## Data Input

**Current Data Availability:**

- Monthly excess returns for **6 anomalies**.
- **Fama–French factors** (MKT, SMB, HML).
- **Regime Classification:**
  - **Pre-Crisis:** 2003-01 to 2007-11
  - **Crisis:** 2007-12 to 2009-06
  - **Post-Crisis:** 2009-07 to 2014-05

---

## Time-Series Regression Analysis (Revised)

This model evaluates whether the risk-adjusted returns of each anomaly changed significantly during the crisis regime, while also accounting for changes in factor exposures (risk).

### Model Specifications

We estimate the following specifications with interaction terms to allow factor loadings to vary by regime.

**Model 1: Pre-crisis vs. Crisis (Baseline: Pre-crisis)**
$$R_t = \alpha_1 + \gamma_1 \cdot \text{Crisis}_t + \beta_1 \cdot \text{Factors}_t + \delta_1 (\text{Crisis}_t \times \text{Factors}_t) + \epsilon_t$$

**Model 2: Crisis vs. Post-crisis (Baseline: Crisis)**
$$R_t = \alpha_2 + \gamma_2 \cdot \text{Post}_t + \beta_2 \cdot \text{Factors}_t + \delta_2 (\text{Post}_t \times \text{Factors}_t) + \epsilon_t$$

### Variable Definitions

- **$R_t$**: The monthly long-short anomaly return.
- **$\text{Factors}_t$**: Vector of Fama–French three factors (MKT, SMB, HML).
- **Dummy Variables**:
  - **$\text{Crisis}_t$**: 1 during the crisis period (Dec 2007–June 2009), 0 otherwise.
  - **$\text{Post}_t$**: 1 during the post-crisis period (July 2009–Dec 2014), 0 otherwise.
- **Coefficients**:
  - **$\beta_1, \beta_2$**: Factor loadings during the respective **baseline** periods (Pre-crisis for Model 1; Crisis for Model 2).
  - **$\gamma_1, \gamma_2$**: Captures the abnormal returns (shift in alpha) during the Crisis and Post-crisis periods, respectively.
  - **$\delta_1, \delta_2$**: Vectors of **interaction coefficients** capturing changes in factor loadings during the Crisis and Post-crisis periods.
- **$\epsilon_t$**: The error term.


In [1]:
import pandas as pd
from IPython.display import display
import statsmodels.api as sm

In [2]:
excess_returns = pd.read_excel('./Regression Data/excess_returns.xlsx', index_col=0)
display(round(excess_returns,3))
excess_returns.info()

,Accruals,Assest Growth,BM,Gross Profit,Momentum,Leaverage Ret,Regime
date,,,,,,,
2003-01-31,-0.007,0.013,-0.004,-0.013,-0.049,0.014,Pre-Crisis
2003-02-28,-0.034,0.009,-0.045,0.022,0.050,-0.006,Pre-Crisis
2003-03-31,-0.039,-0.004,-0.034,-0.012,-0.006,0.001,Pre-Crisis
2003-04-30,0.002,0.049,0.102,-0.110,-0.120,0.007,Pre-Crisis
2003-05-30,0.067,0.077,0.106,-0.171,-0.327,0.002,Pre-Crisis
...,...,...,...,...,...,...,...
2014-01-31,-0.014,0.003,-0.024,-0.144,-0.043,-0.010,Post-Crisis
2014-02-28,-0.008,-0.010,-0.018,0.004,0.010,0.017,Post-Crisis
2014-03-31,0.011,0.040,0.001,0.051,-0.001,0.000,Post-Crisis


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 137 entries, 2003-01-31 to 2014-05-30
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Accruals       137 non-null    float64
 1   Assest Growth  137 non-null    float64
 2   BM             137 non-null    float64
 3   Gross Profit   137 non-null    float64
 4   Momentum       137 non-null    float64
 5   Leaverage Ret  137 non-null    float64
 6   Regime         137 non-null    object 
dtypes: float64(6), object(1)
memory usage: 8.6+ KB


In [3]:
ff_factors = pd.read_excel('./Regression Data/fama_french_factors.xlsx', index_col='date')
display(round(ff_factors.head(),3))
print(ff_factors.info())

,Unnamed: 0,Mkt-RF,SMB,HML,RF,Regime
date,,,,,,
2003-01-31,0,-0.027,0.019,0.008,0.001,Pre-Crisis
2003-02-28,1,-0.018,0.005,0.017,0.001,Pre-Crisis
2003-03-31,2,-0.004,0.005,-0.011,0.001,Pre-Crisis
2003-04-30,3,0.087,-0.000,-0.008,0.001,Pre-Crisis
2003-05-30,4,0.066,0.032,0.005,0.001,Pre-Crisis


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 137 entries, 2003-01-31 to 2014-05-30
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  137 non-null    int64  
 1   Mkt-RF      137 non-null    float64
 2   SMB         137 non-null    float64
 3   HML         137 non-null    float64
 4   RF          137 non-null    float64
 5   Regime      137 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.5+ KB
None


In [4]:
# Merge Data
# We join the anomaly returns with the Fama-French factors.
# We only need Mkt-RF, SMB, and HML from the factors df.
df_combined = pd.merge(
    excess_returns, 
    ff_factors[['Mkt-RF', 'SMB', 'HML']], 
    left_index=True, 
    right_index=True, 
    how='inner'
)
display(round(df_combined.head(),3))

,Accruals,Assest Growth,BM,Gross Profit,Momentum,Leaverage Ret,Regime,Mkt-RF,SMB,HML
date,,,,,,,,,,
2003-01-31,-0.007,0.013,-0.004,-0.013,-0.049,0.014,Pre-Crisis,-0.027,0.019,0.008
2003-02-28,-0.034,0.009,-0.045,0.022,0.050,-0.006,Pre-Crisis,-0.018,0.005,0.017
2003-03-31,-0.039,-0.004,-0.034,-0.012,-0.006,0.001,Pre-Crisis,-0.004,0.005,-0.011
2003-04-30,0.002,0.049,0.102,-0.110,-0.120,0.007,Pre-Crisis,0.087,-0.000,-0.008
2003-05-30,0.067,0.077,0.106,-0.171,-0.327,0.002,Pre-Crisis,0.066,0.032,0.005


In [5]:
# List of anomaly columns (Dependent Variables)
anomalies = ['Accruals', 'Assest Growth', 'BM', 'Gross Profit', 'Momentum', 'Leaverage Ret']

In [ ]:
# # ---  Define the Regression Function ---

# def run_regressions(data, anomalies, dummy_col_name):
#     results_list = []
    
#     # Independent Variables: Constant + 3 Factors + Regime Dummy
#     X_cols = ['Mkt-RF', 'SMB', 'HML', dummy_col_name]
#     X = sm.add_constant(data[X_cols])
    
#     for anomaly in anomalies:
#         # Dependent Variable
#         y = data[anomaly]
        
#         # Fit OLS
#         model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':3})
        
#         # Store results
#         results_list.append({
#             'Anomaly': anomaly,
#             'Gamma_Coef': model.params[dummy_col_name],
#             'P_Value': model.pvalues[dummy_col_name],
#             'Significant (5%)': model.pvalues[dummy_col_name] < 0.05,
#             't_stat': model.tvalues[dummy_col_name],
#             'Beta MKT': model.params['Mkt-RF'],
#             'Beta SMB': model.params['SMB'],
#             'Beta HML': model.params['HML'],
#             'Alpha': model.params['const'],
#             'Adjusted R2': model.rsquared_adj,
#         })
        
#     return pd.DataFrame(results_list)


In [6]:
# --- Updated Regression Function with Interactions ---
def run_regressions_with_interactions(data, anomalies, dummy_col_name):
    results_list = []
    
    # Define Factor columns
    factors = ['Mkt-RF', 'SMB', 'HML']
    
    # Create Interaction Terms: Dummy * Factor
    # This captures the change in factor loading during the specific regime (delta coefficients)
    interaction_cols = []
    for f in factors:
        int_col = f"{dummy_col_name}_x_{f}"
        data[int_col] = data[dummy_col_name] * data[f]
        interaction_cols.append(int_col)
    
    # Independent Variables: Constant + Factors + Dummy + Interactions
    X_cols = factors + [dummy_col_name] + interaction_cols
    X = sm.add_constant(data[X_cols])
    
    for anomaly in anomalies:
        # Dependent Variable
        y = data[anomaly]
        
        # Fit OLS
        model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags':3})
        
        # Store results
        # We now capture the base betas (beta1/beta2) and the interaction deltas (delta1/delta2)
        results_list.append({
            'Anomaly': anomaly,
            'Alpha': model.params['const'],
            
            # Regime Dummy Coefficient (Gamma)
            'Gamma_Coef': model.params[dummy_col_name],
            'Gamma_P_Value': model.pvalues[dummy_col_name],
            'Gamma_t_stat': model.tvalues[dummy_col_name],
            
            # Base Factor Loadings (Beta)
            'Base_Beta_MKT': model.params['Mkt-RF'],
            'Base_Beta_SMB': model.params['SMB'],
            'Base_Beta_HML': model.params['HML'],
            
            # Interaction Coefficients (Delta) - Change in loadings
            'Delta_MKT': model.params[f"{dummy_col_name}_x_Mkt-RF"],
            'Delta_SMB': model.params[f"{dummy_col_name}_x_SMB"],
            'Delta_HML': model.params[f"{dummy_col_name}_x_HML"],
            
            'Interaction_MKT_Pval': model.pvalues[f"{dummy_col_name}_x_Mkt-RF"],
            
            'Adjusted R2': model.rsquared_adj,
        })
        
    return pd.DataFrame(results_list)

In [ ]:

# # --- Run Model 1: Pre-Crisis vs. Crisis ---
# # Filter: Keep only Pre-Crisis and Crisis rows
# df_m1 = df_combined[df_combined['Regime'].isin(['Pre-Crisis', 'Crisis'])].copy()
# print(df_m1['Regime'].unique())

# # Dummy: 1 if Crisis, 0 if Pre-Crisis
# df_m1['CrisisDummy'] = (df_m1['Regime'] == 'Crisis').astype(int)

# results_m1 = run_regressions(df_m1, anomalies, 'CrisisDummy')
# results_m1.to_excel('./Regression Results/model_1_results.xlsx', index=False)
# display(round(results_m1,3))



In [9]:
# --- Run Revised Model 1: Pre-Crisis vs. Crisis ---
# Specification: R = alpha + gamma*Crisis + beta*Factors + delta(Crisis*Factors) + e

# Filter: Keep only Pre-Crisis (Baseline) and Crisis rows
df_m1 = df_combined[df_combined['Regime'].isin(['Pre-Crisis', 'Crisis'])].copy()

# Dummy: 1 if Crisis, 0 if Pre-Crisis
df_m1['CrisisDummy'] = (df_m1['Regime'] == 'Crisis').astype(int)

print("Running Model 1 (Pre-Crisis vs. Crisis) with Interactions...")
results_m1 = run_regressions_with_interactions(df_m1, anomalies, 'CrisisDummy')

results_m1.to_excel('./Regression results with interaction term/model_1_revised_results.xlsx', index=False)
display(round(results_m1, 3))

Running Model 1 (Pre-Crisis vs. Crisis) with Interactions...


,Anomaly,Alpha,Gamma_Coef,Gamma_P_Value,Gamma_t_stat,Base_Beta_MKT,Base_Beta_SMB,Base_Beta_HML,Delta_MKT,Delta_SMB,Delta_HML,Interaction_MKT_Pval,Adjusted R2
0,Accruals,0.001,-0.009,0.238,-1.179,0.079,0.227,-0.442,-0.039,-0.106,-1.222,0.780,0.284
1,Assest Growth,-0.008,-0.002,0.761,-0.304,0.468,0.194,0.492,-0.728,0.076,-0.002,0.000,0.214
2,BM,-0.003,0.022,0.054,1.924,0.896,0.480,0.657,-0.581,-0.353,0.051,0.005,0.449
3,Gross Profit,0.002,0.006,0.394,0.852,-0.656,-0.172,0.385,0.435,-1.451,-0.005,0.092,0.288
4,Momentum,0.006,-0.054,0.196,-1.294,-0.559,-0.570,0.485,-0.070,-0.459,-1.786,0.913,0.137
5,Leaverage Ret,-0.001,0.009,0.270,1.104,-0.046,0.019,-0.056,-0.012,-0.795,0.680,0.922,0.299


In [ ]:
# # --- Run Model 2: Crisis vs. Post-Crisis ---
# # Filter: Keep only Crisis and Post-Crisis rows
# df_m2 = df_combined[df_combined['Regime'].isin(['Crisis', 'Post-Crisis'])].copy()
# print(df_m2['Regime'].unique())
# # Dummy: 1 if Post-Crisis, 0 if Crisis
# df_m2['PostCrisisDummy'] = (df_m2['Regime'] == 'Post-Crisis').astype(int)

# print("\nRunning Model 2 (Crisis vs. Post-Crisis)...")
# results_m2 = run_regressions(df_m2, anomalies, 'PostCrisisDummy')
# results_m2.to_excel('./Regression Results/model_2_results.xlsx', index=False)
# display(round(results_m2,3))

In [10]:
# --- Run Revised Model 2: Crisis vs. Post-Crisis ---
# Specification: R = alpha + gamma*Post + beta*Factors + delta(Post*Factors) + e

# Filter: Keep only Crisis (Baseline) and Post-Crisis rows
df_m2 = df_combined[df_combined['Regime'].isin(['Crisis', 'Post-Crisis'])].copy()

# Dummy: 1 if Post-Crisis, 0 if Crisis
df_m2['PostCrisisDummy'] = (df_m2['Regime'] == 'Post-Crisis').astype(int)

print("\nRunning Model 2 (Crisis vs. Post-Crisis) with Interactions...")
results_m2 = run_regressions_with_interactions(df_m2, anomalies, 'PostCrisisDummy')

results_m2.to_excel('./Regression results with interaction term/model_2_revised_results.xlsx', index=False)
display(round(results_m2, 3))


Running Model 2 (Crisis vs. Post-Crisis) with Interactions...


,Anomaly,Alpha,Gamma_Coef,Gamma_P_Value,Gamma_t_stat,Base_Beta_MKT,Base_Beta_SMB,Base_Beta_HML,Delta_MKT,Delta_SMB,Delta_HML,Interaction_MKT_Pval,Adjusted R2
0,Accruals,-0.007,0.007,0.348,0.938,0.040,0.120,-1.664,0.137,0.012,1.562,0.231,0.351
1,Assest Growth,-0.010,0.009,0.191,1.308,-0.259,0.270,0.490,0.481,-0.135,0.396,0.000,0.321
2,BM,0.019,-0.022,0.059,-1.888,0.314,0.127,0.708,-0.187,0.561,0.494,0.228,0.324
3,Gross Profit,0.008,-0.007,0.391,-0.857,-0.221,-1.623,0.381,0.104,0.941,-0.876,0.351,0.238
4,Momentum,-0.048,0.058,0.165,1.390,-0.629,-1.030,-1.301,0.582,-0.706,-0.091,0.248,0.220
5,Leaverage Ret,0.008,-0.008,0.302,-1.031,-0.058,-0.776,0.624,0.046,0.699,-0.495,0.689,0.244
